In [1]:
import torch, os
import numpy as np
from importlib import reload
os.chdir("/home/gibson/sdb2/cola/ass/balls/2019-hackathon-ocr-wymbah")

In [2]:
import nets.ctcnet
reload(nets.ctcnet)
from nets.ctcnet import CTCModel

import util.beam
reload(util.beam)
from util.beam import BeamSearch

from util.file import FileHolder
fh = FileHolder()

from util.chars import chars, nchars, idx

Using path ../AI4Good---Meza-OCR-Challenge


In [4]:
def to_tensor(buf):
  buf -= 0.5
  buf *= 2
  #buf -= np.mean(buf, axis=(0, 1, 2))[None, None, None, :]
  #buf /= np.std(buf, axis=(0, 1, 2))[None, None, None, :]
  #buf -= np.array([0.485, 0.456, 0.406])[None, None, None, :]
  #buf /= np.array([0.229, 0.224, 0.225])[None, None, None, :]
  return torch.tensor(buf, dtype=torch.float32).permute(0, 3, 1, 2)

def get_batch(batchsize, validation=False):
  inp, outp = fh.get_batch(batchsize, validation)
  inp = np.stack(inp)
  inp = to_tensor(inp)
  return inp, outp

ctcmodel = CTCModel().cuda()
checkpoint = torch.load("checkpoint-0028-0.62-CNNModel")
ctcmodel.load_state_dict(checkpoint['ctcmodel'])
ctcmodel.eval()

def getone():
    inp, outp = get_batch(1, validation=True)
    log = ctcmodel(inp)
    log = log.detach().cpu().numpy()
    return inp, outp, log

import matplotlib.pyplot as p

def show(inp, outp):
    print(outp[0])
    p.imshow(inp.squeeze(0).permute(1, 2, 0))
    p.show()

In [5]:
def beam(log, nbeams):
    beas = BeamSearch(nbeams)
    for j in range(log.shape[1]):
        beas.add_logit(log[0, j, :])
    return beas.topbeams()

In [6]:
def tets():
    inp, outp, log = getone()
    answer = beam(log, 8)
    if answer[0].str() != outp[0]:
        print("beams:", answer)
        print("correct:", outp)
        return 0
    else:
        return 1

In [7]:
tets()

1

In [59]:
outp

['147']